# Sistema de Análise de Ondas de Calor em Python
Desenvolva um sistema que identifica e analisa ondas de calor usando dados históricos de temperatura. O código deve:
- Detectar períodos onde a temperatura máxima excede o percentil 90 por 3+ dias consecutivos
- Calcular intensidade, duração e frequência das ondas de calor
- Gerar mapas de calor temporal, gráficos de tendências decadais e relatórios de impacto climático
- Implementar algoritmos de detecção baseados em índices como HWMId (Heat Wave Magnitude Index daily)

## Importação das bibliotecas

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import calendar
from datetime import datetime, timedelta


## Configurações iniciais

In [3]:
plt.style.use('ggplot')
sns.set_palette("YlOrRd")

## Desenvolvimento da classe

In [4]:
class HeatWaveAnalyzer:
    def __init__(self, temp_data, threshold_percentile=90, min_consecutive_days=3):
        """
        Inicializa o analisador de ondas de calor.
        
        Args:
            temp_data (DataFrame): Dados de temperatura com colunas 'date' e 'temp_max'
            threshold_percentile (int): Percentil para definir o limiar de onda de calor (padrão: 90)
            min_consecutive_days (int): Mínimo de dias consecutivos para considerar onda de calor (padrão: 3)
        """
        self.data = temp_data.copy()
        self.threshold_percentile = threshold_percentile
        self.min_consecutive_days = min_consecutive_days
        self.heat_waves = None
        self.climate_report = None

        # Pré-processamento
        self._preprocess_data()
    
    def _preprocess_data(self):
        """Prepara os dados para analise."""
        # Garante que a data está no formato correto
        self.data['date'] = pd.to_datetime(self.data['date'])
        self.data = self.data.sort_values('date').reset_index(drop=True)

        # Adiciona colunas auxiliares
        self.data['year'] = self.data['date'].dt.year
        self.data['month'] = self.data['date'].dt.month
        self.data['day'] = self.data['date'].dt.day
        self.data['day_of_year'] = self.data['date'].dt.dayofyear

        # Calcula o limiar de temperatura para cada dia do ano (percentil movel)
        self._calculate_thresholds()